In [ ]:
import datetime
import math
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import pickle

import optuna
import lightgbm as lgb

from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
# import tensorflow_addons as tfa
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
import numpy as np

import time

In [ ]:
with open('inverter-data-cnn-v01.pkl', 'rb') as handle:
    x_dict, y_dict, label_df = pickle.load(handle)

In [ ]:
x_all, y_all = [], []
for inv in x_dict:
    x_ii, y_ii = x_dict[inv], y_dict[inv]
    x_all.append(x_ii)
    y_all.append(y_ii)

x_all = np.concatenate(x_all, axis=0)
y_all = np.concatenate(y_all, axis=0)
x_all.shape, y_all.shape

In [ ]:
# y_all = np.expand_dims(y_all, axis=-1)
# y_all = tf.one_hot(y_all, depth=2)
# y_all.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=100)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=100)

In [ ]:
y_train = tf.one_hot(y_train, depth=2)
y_val = tf.one_hot(y_val, depth=2)
y_test = tf.one_hot(y_test, depth=2)
y_train.shape, y_val.shape, y_test.shape

In [ ]:
class Classifier_RESNET:

    def __init__(self, output_directory, input_shape, nb_classes, batch_size,
                 lr, epochs, verbose=False, build=True, load_weights=False):
        self.output_directory = output_directory
        self.batch_size = batch_size
        self.lr = lr
        self.epochs = epochs
        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
            if load_weights == True:
                self.model.load_weights(self.output_directory
                                        .replace('resnet_augment', 'resnet')
                                        .replace('TSC_itr_augment_x_10', 'TSC_itr_10')
                                        + '/model_init.hdf5')
            else:
                self.model.save_weights(self.output_directory + 'model_init.hdf5')
        return

    def build_model(self, input_shape, nb_classes):
        n_feature_maps = 64

        input_layer = keras.layers.Input(input_shape)

        # BLOCK 1

        conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_1 = keras.layers.add([shortcut_y, conv_z])
        output_block_1 = keras.layers.Activation('relu')(output_block_1)

        # BLOCK 2

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_2 = keras.layers.add([shortcut_y, conv_z])
        output_block_2 = keras.layers.Activation('relu')(output_block_2)

        # BLOCK 3

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # no need to expand channels because they are equal
        shortcut_y = keras.layers.BatchNormalization()(output_block_2)

        output_block_3 = keras.layers.add([shortcut_y, conv_z])
        output_block_3 = keras.layers.Activation('relu')(output_block_3)

        # FINAL

        gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)

        file_path = self.output_directory + 'best_model.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                           save_best_only=True)

        self.callbacks = [reduce_lr, model_checkpoint]

        return model

    def fit(self, x_train, y_train, x_val, y_val):
        # if not tf.test.is_gpu_available:
        #     print('error')
        #     exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training
        # batch_size = 64
        # nb_epochs = 1500

        # mini_batch_size = int(min(x_train.shape[0] / 10, batch_size))

        start_time = time.time()

        hist = self.model.fit(x_train, y_train,
                              batch_size=self.batch_size,
                              epochs=self.epochs,
                              verbose=self.verbose,
                              validation_data=(x_val, y_val),
                              callbacks=self.callbacks)

        duration = time.time() - start_time

        self.model.save(self.output_directory + 'last_model.hdf5')
        train_pred = self.model.predict(x_train, batch_size=self.batch_size)
        test_pred = self.model.predict(x_val, batch_size=self.batch_size)

        # y_pred = self.predict(x_val, y_true, x_train, y_train, y_val,
        #                       return_df_metrics=False)

        # save predictions
        # np.save(self.output_directory + 'y_pred.npy', y_pred)

        # convert the predicted from binary to integer
        # y_pred = np.argmax(y_pred, axis=1)

        # df_metrics = save_logs(self.output_directory, hist, y_pred, y_true, duration)

        keras.backend.clear_session()

        return hist, train_pred, test_pred

    def predict(self, x_test, y_true, x_train, y_train, y_test, return_df_metrics=True):
        start_time = time.time()
        model_path = self.output_directory + 'best_model.hdf5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test)
        if return_df_metrics:
            y_pred = np.argmax(y_pred, axis=1)
            df_metrics = calculate_metrics(y_true, y_pred, 0.0)
            return df_metrics
        else:
            test_duration = time.time() - start_time
            save_test_duration(self.output_directory + 'test_duration.csv', test_duration)
            return y_pred


In [ ]:
_, seq_len, num_features = x_train.shape
cls = Classifier_RESNET(output_directory='./', 
                        input_shape=(seq_len, num_features),
                        nb_classes=2,
                        batch_size=32,
                        lr=1e-05,
                        epochs=50,
                        verbose=True,
                        build=True,
                        load_weights=False)

hist, train_pred, val_pred = cls.fit(x_train, y_train, x_val, y_val)


In [ ]:
print(hist.history.keys())
# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss - MSE')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

# summarize history for additional metric
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()



In [ ]:
class Classifier_INCEPTION:

    def __init__(self, 
                 output_directory,
                 input_shape,
                 nb_classes, 
                 batch_size=64,
                 lr=0.001,
                 nb_epochs=1500,
                 verbose=False,
                 build=True,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41):

        self.output_directory = output_directory

        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.batch_size = batch_size
        self.bottleneck_size = 32
        self.nb_epochs = nb_epochs
        self.lr = lr
        self.verbose = verbose

        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if (verbose == True):
                self.model.summary()
#             self.model.save_weights(self.output_directory + 'model_init.hdf5')

    def _inception_module(self, input_tensor, stride=1, activation='linear'):

        if self.use_bottleneck and int(input_tensor.shape[-1]) > self.bottleneck_size:
            input_inception = keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = keras.layers.Concatenate(axis=2)(conv_list)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation(activation='relu')(x)
        return x

    def _shortcut_layer(self, input_tensor, out_tensor):
        shortcut_y = keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                         padding='same', use_bias=False)(input_tensor)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        x = keras.layers.Add()([shortcut_y, out_tensor])
        x = keras.layers.Activation('relu')(x)
        return x

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        x = input_layer
        input_res = input_layer

        for d in range(self.depth):

            x = self._inception_module(x)

            if self.use_residual and d % 3 == 2:
                x = self._shortcut_layer(input_res, x)
                input_res = x

        gap_layer = keras.layers.GlobalAveragePooling1D()(x)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(self.lr),
                      metrics=['accuracy'])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5,
                                                      min_lr=1e-07)

        file_path = self.output_directory + 'best_model.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                           save_best_only=True)

        self.callbacks = [reduce_lr, model_checkpoint]

        return model

    def fit(self, x_train, y_train, x_val, y_val):
#         if not tf.test.is_gpu_available:
#             print('error no gpu')
#             exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training

        if self.batch_size is None:
            mini_batch_size = int(min(x_train.shape[0] / 10, 16))
        else:
            mini_batch_size = self.batch_size

        start_time = time.time()

        hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=self.nb_epochs,
                              verbose=self.verbose, validation_data=(x_val, y_val), callbacks=self.callbacks)

        duration = time.time() - start_time

        self.model.save(self.output_directory + 'last_model.hdf5')
        train_pred = self.model.predict(x_train, batch_size=self.batch_size)
        val_pred = self.model.predict(x_val, batch_size=self.batch_size)

#         y_pred = self.predict(x_val, y_true, x_train, y_train, y_val,
#                               return_df_metrics=False)

        # save predictions
#         np.save(self.output_directory + 'y_pred.npy', y_pred)

        # convert the predicted from binary to integer
#         y_pred = np.argmax(y_pred, axis=1)

#         df_metrics = save_logs(self.output_directory, hist, y_pred, y_true, duration)

        keras.backend.clear_session()

        return hist, train_pred, val_pred

    def predict(self, x_test, y_true, x_train, y_train, y_test, return_df_metrics=True):
        start_time = time.time()
        model_path = self.output_directory + 'best_model.hdf5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test, batch_size=self.batch_size)
        if return_df_metrics:
            y_pred = np.argmax(y_pred, axis=1)
            df_metrics = calculate_metrics(y_true, y_pred, 0.0)
            return df_metrics
        else:
            test_duration = time.time() - start_time
            save_test_duration(self.output_directory + 'test_duration.csv', test_duration)
            return y_pred


In [ ]:
BATCH, base_lr, EPOCHS = 32, 1e-05, 50
cls = Classifier_INCEPTION(output_directory='./', 
                    input_shape=(seq_len, num_features), 
                    nb_classes=2, 
                    batch_size=BATCH,
                    lr=base_lr,
                    nb_epochs=EPOCHS,
                    verbose=True, 
                    build=True,
                    nb_filters=32, 
                    use_residual=True, 
                    use_bottleneck=True, 
                    depth=6, 
                    kernel_size=41, 
#                     lr_scheduler=scheduler
                          )
hist, train_pred, val_pred = cls.fit(x_train, y_train, x_val, y_val)

In [ ]:
print(hist.history.keys())
# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss - MSE')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

# summarize history for additional metric
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

In [ ]:
test_pred = cls.model.predict(x_test, batch_size=BATCH)

In [ ]:
train_auc = roc_auc_score(y_true=y_train, y_score=train_pred)
val_auc = roc_auc_score(y_true=y_val, y_score=val_pred)
test_auc = roc_auc_score(y_true=y_test, y_score=test_pred)
print(f"train-auc: {train_auc}, val-auc: {val_auc}, test-auc: {test_auc}")

In [ ]:
pos_label = 1
# y_true = np.argmax(y_test, axis=1)
average_precision = average_precision_score(y_test, test_pred)
precision, recall, thresholds = precision_recall_curve(np.argmax(y_test, axis=1), test_pred[:,1])

line_kwargs = {"drawstyle": "steps-post"}
# line_kwargs = {}
line_kwargs["label"] = (f"AP = {average_precision:0.2f}")

plt.figure(figsize=(10, 5))
sns.set(font_scale = 1)
fig, ax = plt.subplots()
ax.plot(recall, precision, **line_kwargs)
ax.plot(recall[:-1], thresholds, **line_kwargs)
info_pos_label = (f" (Positive label: {pos_label})"
                  if pos_label is not None else "")
xlabel = "Recall" + info_pos_label
ylabel = "Precision" + info_pos_label
ax.set(xlabel=xlabel, ylabel=ylabel)
ax.legend(loc="lower left")

## Data Preparation for CNN Models
    - extract past 1d/2d/7d etc. history

In [ ]:
with open('inverter-data-v03.pkl', 'rb') as handle:
    all_data = pickle.load(handle)
    
print(all_data.shape)
all_data.dropna(inplace=True)
all_data.shape

In [ ]:
# filter for hourly data
all_data = all_data[all_data.date.dt.minute==0].copy()

# filter for daily data
all_data = all_data[all_data.date.dt.hour==6]

all_data.shape

In [ ]:
# label_col = 'label_1h'
label_col = 'label_24h'

label_df = all_data[['date', 'inverter', label_col]]

In [ ]:
label_df[label_col].value_counts()

In [ ]:
del all_data

In [ ]:
inv_df = pd.read_csv('all_inverters.csv')

In [ ]:
window = 1  # in days
base_features = [
                'IN.GMRX.CHAR.WS-20 MW.Module Temperature (°C)',
                'IN.GMRX.CHAR.WS-20 MW.POA Irradiance (w/m²)',
                'IN.GMRX.CHAR.WS-5 MW.Module Temperature (°C)',
                'IN.GMRX.CHAR.WS-5 MW.POA Irradiance (w/m²)'
               ]
dfg = label_df.groupby('inverter')
x, y = dict(), dict()
for inverter, df in dfg:
    features = ['IN.GMRX.CHAR.'+inverter+'.Active Power (kW)'] + base_features
    columns = ['date'] + features
    inv_df_i = inv_df[columns].copy()
    inv_df_i['date'] = pd.to_datetime(inv_df_i["date"])
    inv_df_i.rename(columns={'IN.GMRX.CHAR.'+inverter+'.Active Power (kW)': 'power',
                            'IN.GMRX.CHAR.WS-20 MW.Module Temperature (°C)': 'temp1',
                            'IN.GMRX.CHAR.WS-20 MW.POA Irradiance (w/m²)': 'rad1',
                            'IN.GMRX.CHAR.WS-5 MW.Module Temperature (°C)': 'temp2',
                            'IN.GMRX.CHAR.WS-5 MW.POA Irradiance (w/m²)': 'rad2'}, inplace=True)
    df = df.reset_index(drop=True)
    x_inv, y_inv = [], []
    for jj, row in df.iterrows():
        end = row['date']
        start = end - pd.Timedelta(window, 'D')
        x_jj = inv_df_i[(inv_df_i.date >  start) & (inv_df_i.date <= end)][['power', 'temp1', 'rad1']].values
        y_jj = row[label_col]
        x_inv.append(x_jj)
        y_inv.append(y_jj)
    x_inv = np.stack(x_inv, axis=0)
    y_inv = np.array(y_inv)
    x[inverter] = x_inv
    y[inverter] = y_inv
    print(inverter, x_inv.shape, y_inv.shape)

In [ ]:
data = (x, y, label_df)
with open('inverter-data-cnn-daily.pkl', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
data = (x, y, label_df)
with open('inverter-data-cnn-v01.pkl', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)